# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки я буду сравнивать поведение пользователей двух столиц.

**Цель исследования** — проверю три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей я получаю из файла `yandex_music_project.csv`. О качестве данных мне ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Я проверю данные на ошибки и оценю их влияние на исследование. Затем, на этапе предобработки я буду искать возможность исправить самые критичные ошибки данных.
 
Таким образом, мое исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составление первого представления о данных Яндекс.Музыки.




Мой основной инструмент — `pandas`. Я импортирую эту библиотеку.

In [ ]:
import pandas as pd # импорт библиотеки pandas

Осуществляю чтение файла `yandex_music_project.csv` из папки `/datasets` и сохраняю его в переменной `df`:

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv') # чтение файла с данными и сохранение в df

Вывожу на экран первые десять строк таблицы:

In [ ]:
display(df.head(10)) # получение первых 10 строк таблицы df

Получаю общую информацию о таблице, с которой мне предстоит работать:

In [ ]:
df.info() # получение общей информации о данных в таблице df

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Составное имя переменной написано без знака подчёркивания - слитно.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, сначала я устраню проблемы в данных.

## Предобработка данных
Исправление стиля в заголовках столбцов, исключение пропусков. Проверка данных на дубликаты.

### Стиль заголовков
Вывожу на экран названия столбцов:

In [ ]:
#print(df.columns) # перечень названий столбцов таблицы df
df.columns

Привожу названия в соответствие с хорошим стилем:
* несколько слов в названии записываю в «змеином_регистре»,
* все символы делаю строчными,
* устраняю пробелы.

Для этого переименовываю колонки следующим образом:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [ ]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})
# переименование столбцов

Проверяю результат. Для этого ещё раз вывожу на экран названия столбцов:

In [ ]:
print(df.columns) # проверка результатов - перечень названий столбцов

### Пропуски значений
Для начала считаю, сколько в таблице пропущенных значений. Для этого достаточно использую два метода `pandas` - isna( ) и sum( ):

In [ ]:
print(df.isna().sum()) # подсчёт пропусков

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для нашего исследования. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Но пока такой возможности нет, поэтому будут проведены следующие действия:
* заполнены и эти пропуски явными обозначениями,
* проведена оценка, насколько эта замена повредит расчётам. 

Заменяю пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создаю список `columns_to_replace`, перебираю его элементы циклом `for` и для каждого столбца выполняю замену пропущенных значений:

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']

for element in columns_to_replace:
    df[element] = df[element].fillna('unknown')

# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

Провожу проверку, что в таблице не осталось пропусков. Для этого ещё раз подсчитываю пропущенные значения.

In [ ]:
print(df.isna().sum())
# повторный подсчёт пропусков после выполнения замены значений (вернет нули при успешной замене)

### Дубликаты
Подсчет явных дубликатов в таблице:

In [ ]:
df.duplicated().sum() # подсчёт явных дубликатов
df.info()

Вызываю специальный метод `pandas` drop_duplicates( ) для удаления явных дубликатов:

In [16]:
df = df.drop_duplicates().reset_index(drop = True)
# удаление явных дубликатов (с удалением старых индексов и формированием новых)

Ещё раз рассчитываю явные дубликаты в таблице для подтверждения корректности удаления:

In [17]:
df.duplicated().sum()
# проверка на отсутствие дубликатов после удаления (вернет ноль при успешном удалении)

0

Теперь будем избавляться от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже могут повлиять на результат моего исследования.

Вывожу на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлекаю нужный столбец датафрейма, 
* применяю к нему метод сортировки,
* для отсортированного столбца вызываю метод unique( ), который вернёт уникальные значения из столбца.

In [18]:
genres_list = df['genre'].sort_values().unique()
print(genres_list) # Просмотр уникальных названий жанров

def find_genre(genre):
    count = 0
    for i in genres_list:
        if i == genre:
            count = count + 1
    if count == 0:
        print('Треки в жанре', genre, 'не обнаружены')
    else:
        print('Треки в жанре', genre, 'присутствуют')
print()
find_genre('hip')
find_genre('hop')
find_genre('hip-hop')
# Проверка на неявные дубликаты

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

Просматривая полученный список, нахожу неявные дубликаты названия `hiphop`. В данном случае это альтернативные названия этого жанра - неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, пишу функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция будет исправлять колонку `genre` в таблице `df` и заменять каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [19]:
def replace_wrong_genres(wrong_genres, correct_genre):    
    for element in wrong_genres:
        df['genre'] = df['genre'].replace(element, correct_genre)
# Функция для замены неявных дубликатов

Вызываю `replace_wrong_genres()` и передаю ей аргументы, необходимые для устранения неявных дубликатов: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [20]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')
# Устранение неявных дубликатов

После выполнения функции снова вывожу список уникальных значений столбца `genre` и проверяю, что неправильные названия в исходной таблице более не обнаруживаются:

In [21]:
genres_list = df['genre'].sort_values().unique()
print(genres_list) # Просмотр уникальных названий жанров
print()

find_genre('hip')
find_genre('hop')
find_genre('hip-hop')
# Проверка на неявные дубликаты

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 'folklore' 'folkmetal'
 'folkrock' 

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Были исправлены заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменены на `'unknown'`. Ещё предстоит проанализировать, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Буду проверять это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделю пользователей Москвы и Санкт-Петербурга
* Сравню, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Сначала выполню каждый из расчётов по-отдельности. 

Оценю активность пользователей в каждом городе. Сгруппирую данные по городу и посчитаю прослушивания в каждой группе.

In [22]:
df.groupby('city')['user_id'].count()
# Подсчёт прослушиваний в каждом городе

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппирую данные по дню недели и подсчитаю прослушивания в понедельник, среду и пятницу.


In [23]:
df.groupby('day')['user_id'].count()
# Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создаю функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраняю в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого применяю последовательную фильтрацию с логической индексацией.

Затем рассчитываю значения в столбце `user_id` получившейся таблицы. Результат сохраняю в новую переменную. В качестве результата выполнения функции возвращаю эту переменную.

In [24]:
def number_tracks(day, city):
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

# Функция для подсчёта прослушиваний для конкретного города и дня

Вызываю `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [25]:
mon_msk = number_tracks('Monday', 'Moscow')
print('Количество прослушиваний в Москве по понедельникам -', mon_msk)
# количество прослушиваний в Москве по понедельникам

Количество прослушиваний в Москве по понедельникам - 15740


In [26]:
mon_spb = number_tracks('Monday', 'Saint-Petersburg')
print('Количество прослушиваний в Санкт-Петербурге по понедельникам -', mon_spb)

Количество прослушиваний в Санкт-Петербурге по понедельникам - 5614


In [27]:
wed_msk = number_tracks('Wednesday', 'Moscow')
print('Количество прослушиваний в Москве по средам -', wed_msk)

Количество прослушиваний в Москве по средам - 11056


In [28]:
wed_spb = number_tracks('Wednesday', 'Saint-Petersburg')
print('Количество прослушиваний в Санкт-Петербурге по средам -', wed_spb)

Количество прослушиваний в Санкт-Петербурге по средам - 7003


In [29]:
fri_msk = number_tracks('Friday', 'Moscow')
print('Количество прослушиваний в Москве по пятницам -', fri_msk)

Количество прослушиваний в Москве по пятницам - 15945


In [30]:
fri_spb = number_tracks('Friday', 'Saint-Petersburg')
print('Количество прослушиваний в Санкт-Петербурге по пятницам -', fri_spb)

Количество прослушиваний в Санкт-Петербурге по пятницам - 5895


Создаю c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые я получила с помощью функции `number_tracks`.

In [31]:
music_column = ['city', 'monday', 'wednesday', 'friday']
#music_data = [['Moscow', 15740, 11056, 15945], ['Saint-Petersburg', 5614, 7003, 5895]]

music_data = [['Moscow', mon_msk, wed_msk, fri_msk], ['Saint-Petersburg', mon_spb, wed_spb, fri_spb]]

table = pd.DataFrame(data = music_data, columns = music_column)
display(table)
# Таблица с результатами

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраняю таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [32]:
moscow_general = df[df['city'] == 'Moscow']

In [33]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Создаю функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [34]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)
    return genre_df_count

# Объявление функции genre_weekday(), которая возвращает информацию
# о 10 самых популярных жанрах за указанный день за заданное время)

Cравниваю результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [35]:
print()
print('Данные за утро по Москве')
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')


Данные за утро по Москве


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [36]:
print()
print('Данные за утро по Санкт-Петербургу')
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)


Данные за утро по Санкт-Петербургу


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [37]:
print()
print('Данные за вечер по Москве')
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')
# вызов функции для вечера пятницы в Москве


Данные за вечер по Москве


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [38]:
print()
print('Данные за утро по Санкт-Петербургу')
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')
# вызов функции для вечера пятницы в Петербурге


Данные за утро по Санкт-Петербургу


genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппирую таблицу `moscow_general` по жанру и посчитаю прослушивания треков каждого жанра методом `count()`. Затем отсортирую результат в порядке убывания и сохраню его в таблице `moscow_genres`.

In [39]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)
# подсчёт количества прослушиваний треков каждого жанра в Москве

Вывожу на экран первые десять строк `moscow_genres`:

In [40]:
print(moscow_genres.head(10))
# просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


Теперь повторю то же и для Санкт-Петербурга.

Сгруппирую таблицу `spb_general` по жанру. Посчитаю прослушивания треков каждого жанра. Результат отсортирую в порядке убывания и сохраню в таблице `spb_genres`:


In [41]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
# подсчёт количества прослушиваний треков каждого жанра в Санкт-Петербурге

Вывожу на экран первые десять строк `spb_genres`:

In [42]:
print(spb_genres.head(10))
# просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Я проверила три гипотезы и установила следующее:

**Гипотеза №1.** День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

**Гипотеза №2.** Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

**Гипотеза №3.** Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных.